# Faster R-CNN调研报告

# 1.前言
## 1.1关键词
- Object Detection (目标检测)
- Region Proposal (区域建议)
- Convolutional Neural Network (卷积神经网络)

## 1.2效果及目的图
![fig0](fig/fig0.png)
## 1.3算法背景简介
 Faster R-CNN的论文在提出时其效果优于当时最好的算法。并且此算法采用的 Region Proposal Networks(RPNs) 使得其计算效率得到很大提升。“在ILSVRC和COCO 2015的竞赛中，Faster R-CNN和RPN是 ImageNet detection,、ImageNet localization、COCO detection和COCO segmentation上获得多项第一名的基础。RPN完全能够学习从数据中提取regions，因此可以很容易地从更深层和更具表现力的特性（如RestNet-101中采用的101层残差网络）中获益。Faster R-CNN和RPN也被这些竞赛中的其他几个领先的参赛者使用。这些结果表明，该方法不仅是一种经济有效的解决方案，而且是一种提高目标检测精度的有效方法。”——原论文

# 2.网络结构（backbone）
![backbone](fig/backbone.jpg)

![fig2](fig/fig2.png)

# 3.算法详解
## 3.1输入
论文中图片输入shape为 M x N = 1000 x 600
## 3.2卷积预处理
论文中提到三种方式以上，其中 VGG-16 和 ZFNet (Zeiler and Fergus model,含5个卷积层) 提及和进行实验较多，作者在4.2节中也提到过用 ResNet-101。

其中使用 VGG-16 的 feature map 维度为 512，ZFNet 的 feature map 为 256，大小均为 60 x 40

## 3.3.Region Proposal Networks(RPNs)
论文就是重点讲解 RPN 的原理及其损失函数
![backbone_signed](fig/backbone_signed.png)
**RPN 的特点**
1. 全卷积网络
2. 与Fast R-CNN共享卷积层
3. 网络结构中存在两个分支，且一个是分类（cls层），一个是回归（reg层）
4. 网络的输出为 Proposal（确定box的4个信息）和其 score，

### 3.3.1 Intersection over Union（IoU）,mAP
IoU是一种测量在特定数据集中检测相应物体准确度的一个标准

计算很简单理解，即预测框和 ground truth 的交和比。

一般来说，这个 score ＞ 0.5 就可以被认为一个不错的结果了。
![fig7](fig/fig7.png)

- 论文中对正负样本的定义：某一类预测框中，IoU 最高的和IoU > 0.7的 anchors为正样本，对所有的ground-truth boxes，满足所有IoU < 0.3的定义为负样本。其余的样本对训练没有作用，标记为-1.（注意到一个ground-truth可以对应多个正样本）

1. True positives(TP):  被正确地划分为正例的个数，即实际为正例且被分类器划分为正例的实例数（样本数）
2. False positives(FP): 被错误地划分为正例的个数，即实际为负例但被分类器划分为正例的实例数
3. False negatives(FN):被错误地划分为负例的个数，即实际为正例但被分类器划分为负例的实例数
4. True negatives(TN): 被正确地划分为负例的个数，即实际为负例且被分类器划分为负例的实例数

![fig6](fig/fig6.jpg)

Average Precision（AP）

Mean Average Precision（mAP），即 平均AP值 

常规的mAP计算为（这是一个N类检测任务）：
1. 算一个mini-batch中class1的精度P（默认IOU大于0.7即为TP）
2. 循环所有测试集图片，重复1过程求所有图片P的均值即为class1的AP
3. 对剩余N-1类重复1和2过程，对每个类的AP求均值，即为mAP

### 3.3.2 Anchors
处理多尺度的问题，使用的一般方法是构建图像金字塔和滤波器金字塔，此论文引用的方法是构建多尺度和横纵比的 anchor
![fig5](fig/fig5.jpg)
anchors可以理解为一些预设大小的框，anchors的种类用k表示，在原文中k=9，由3种面积(128^2,256^2,512^2)和3种长宽比(1:1,1:2,2:1)组成
![fig1](fig/fig1.png)
![fig20](fig/fig20.png)
![fig19](fig/fig19.png)
#### 3.3.2.1创建和理解
注意到开头那个 3 x 3卷积

对于每个3x3窗口，作者假定它来自９种不同原始区域的池化，但是这些池化在原始图片中的中心点都完全一样。

在 feature map 的每个点上创建 9 个 anchors， 

"For a convolutional feature map of a size W × H (typically ∼2,400), there are WHk anchors in total." 

那么一共有 60 x 40 x 9 ∼ 20000 个每个点创建的 anchors 其实都作用于原图像。

但实际上需要忽略超出边界上的 anchors ，剩下约 6000，然后忽略 IoU < 阈值 的 anchors，最后剩下∼2000个。

#### 3.3.2.3 translation invariant(平移不变性)
"An important property of our approach is that it is translation invariant, both in terms of the anchors and the functions that compute proposals relative to the anchors."

此特性还有助于减小模型参数,在小数据集上过拟合风险小。

"(4 + 2) × 9-dimensional convolutional output layer in the case of k = 9 anchors."

"As a result, our output layer has 2.8 × 104 parameters (512 × (4 + 2) × 9 for VGG-16)"
#### 3.3.2.4 多尺度的 anchor 作为回归 References

![fig13](fig/fig13.png)

![fig8](fig/fig8.png)

#### 3.3.2.5 理解cls层和reg层
每个anhcor要分前景(foreground，正例，正样本)和后景(background)，所以每个点由256-d（VGG-16为512-d）feature转化为cls=2k scores；

而每个anchor都有[x, y, w, h]对应4个偏移量，所以reg=4k coordinates（scores和coordinates为RPN的最终输出）

![fig16](fig/fig16.png)

### 3.3.3 Loss Function
![fig3](fig/fig3.png) 
这里的 i 是在一个 mini-batch 里 anchor 的索引，

pi 是预测 i 所对应的anchor为前景(positive)的概率，
 
ground-truth label pi∗ 是 1 表示这个 anchor 是 positive ，0 表示 negative。

ti 是预测的 bounding box 的4维向量，对应参数化的坐标，ti∗ 是 positive 的 anchor 的 ground-truth 对应的坐标。

"The two terms are normalized by Ncls and Nreg and weighted by a balancing parameter λ",

提到归一化操作和平衡化，其中 Ncls = mini-batch size = 256，Nreg = feature map size ∼2400，λ作者给出经验值10。

而 1/256 ∼ 10∗1/2400，结果权重基本相同。

函数项
![fig9](fig/fig9.png) 
![fig10](fig/fig10.png)

其中的R, "where R is the robust loss function (smooth L1)"

![fig12](fig/fig12.png)

### 3.3.4 cls层与reg层的融合

坐标值的精确更新
![fig17](fig/fig17.jpg)
![fig4](fig/fig4.png)

### 3.3.5 训练细节
由于实际上正负样本数量上差距大，论文提出这样一种解决方法

"we randomly sample 256 anchors in an image to compute the loss function of a mini-batch, where the sampled positive and negative anchors have a ratio of up to 1:1."

"If there are fewer than 128 positive samples in an image, we pad the mini-batch with negative ones."

## 3.4 论文实验

论文实验部分中对 300 个 anchors 的共享层 RPN 测试评价很高，之所以可以选取这么少的anchors，论文中证明了是 cls层的作用。

![fig14](fig/fig14.png)

## 3.5 ROI Pooling
![fig21](fig/fig21.png)

输入的是RPN层产生的region proposal(假定有300个region proposal box)和VGG16最后一层产生的特征图(60 x 40 512-d)，遍历每个region proposal，将其坐标值缩小16倍，这样就可以将在原图(1000 x 600)基础上产生的region proposal映射到60 x 40的特征图上，从而将在feature map上确定一个区域(定义为RB*)。
在feature map上确定的区域RB∗，根据参数例如pooled_w:7,pooled_h:7,将这个RB∗区域划分为7 x 7，即49个相同大小的小区域，对于每个小区域，使用max pooling方式从中选取最大的像素点作为输出，这样，就形成了一个7 x 7的feature map。

其实现在用的是 RoIAlign 代替ROI Pooling，很简单，保留浮点数，双线性插值。

## 3.5 全连接层，输出

![fig15](fig/fig15.png)

通过full connect层与softmax计算每个region proposal具体属于哪个类别（如人，马，车等），

输出cls_prob概率向量；同时再次利用bounding box regression获得每个region proposal的位置偏移量bbox_pred，

用于回归获得更加精确的目标检测框

即从RoI Pooling获取到7x7大小的proposal feature maps后，通过全连接主要做了：

1. 通过全连接和softmax对region proposals进行具体类别的分类
2. 再次对region proposals进行bounding box regression，获取更高精度的rectangle box
